In [2]:
import pandas as pd
import ast

In [3]:
emails = pd.read_csv('20231104_human_validation_dataset.csv')

In [4]:
len(emails)

44

In [5]:
emails.columns

Index(['subject', 'sender', 'message', 'replier', 'reply_message'], dtype='object')

In [10]:
emails = emails.dropna()

In [25]:
emails.head()

,subject,sender,message,replier,reply_message,Email_ID,Sender_Receiver_Emails,Sender_Emails_All
0,Basketball Game,Scott,Yoo do you wanna go watch the bball game tomor...,Radhika,"Hey Scott,\n\nI appreciate the invite a lot. B...",1,"[Hey Scott,\n\nAs you already know, I am a veg...","[Hey Scott,\n\nAs you already know, I am a veg..."
1,Basketball Game,Scott,Bro itll be fun trust me. I wanna watch sports...,Radhika,"Hey Scott,\n\nAs you already know, I am a vege...",2,"[Hey Scott,\n\nI appreciate the invite a lot. ...","[Hey Scott,\n\nI appreciate the invite a lot. ..."
2,Basketball Game,Scott,ah but meat is so delish. no worries though. T...,Radhika,"Hello Scott,\n\nI should be free to go at 9pm ...",3,"[Hey Scott,\n\nI appreciate the invite a lot. ...","[Hey Scott,\n\nI appreciate the invite a lot. ..."
3,Basketball Game,Scott,"Ill teach u about basketball dont worry, ill g...",Radhika,"Scott,\n\nThat sounds great! You do owe me a d...",4,"[Hey Scott,\n\nI appreciate the invite a lot. ...","[Hey Scott,\n\nI appreciate the invite a lot. ..."
4,Basketball Game,Scott,I like baseball too. Can you do sports betting...,Radhika,"Hey Scott,\n\nI don't think so - badminton is ...",5,"[Hey Scott,\n\nI appreciate the invite a lot. ...","[Hey Scott,\n\nI appreciate the invite a lot. ..."


In [11]:
emails['Email_ID'] = range(1, len(emails) + 1)

In [14]:
# emails['recipient'] = emails['replier'].apply(ast.literal_eval)

In [26]:
def get_sender_receiver_emails(row):
    sender = row['sender']
    recipient = row['replier']
    email_id = row['Email_ID']
        
    recipient_df = emails[emails.replier == recipient]
    recipient_df = recipient_df[recipient_df.Email_ID != email_id]
    
    top_emails = recipient_df[recipient_df['sender']==sender]    
    
    # if(top_emails.empty):
    #     return ""
    
    relevent_emails = list(top_emails['reply_message'])    
    
    sender_df = emails[emails.replier == sender]
    sender_df = sender_df[sender_df.Email_ID != email_id]
    
    top_emails_2 = sender_df[sender_df['sender']==recipient]    
    
    # if(top_emails.empty):
    #     return ""
    
    relevent_emails = relevent_emails + list(top_emails_2['message'])
    relevent_emails = [str(item) for item in relevent_emails]
#     print(len(relevent_emails))
#     print('\n')
#     combined_string = " ".join(relevent_emails)
#     return combined_string
    return relevent_emails

def get_sender_emails(row):
    sender = row['replier']
    email_id = row['Email_ID']
        
    sender_df = emails[emails.replier == sender]
    sender_df = sender_df[sender_df.Email_ID != email_id]
    
    other_df = emails[emails.sender == sender]
    other_df = other_df[other_df.Email_ID != email_id]

    if(sender_df.empty):
        return ""
    
    relevent_emails = list(sender_df['reply_message'])
    
    relevent_emails = relevent_emails + list(other_df['message'])
    
    relevent_emails = [str(item) for item in relevent_emails]
    
#     print(len(relevent_emails))
#     print('\n')
    
#     combined_string = " ".join(relevent_emails)
    
#     return combined_string
    return relevent_emails

def remove_same_emails(row):
    all_emails = row['Sender_Emails_All']
    sender_receiver_emails = row['Sender_Receiver_Emails']    
    
    return [x for x in all_emails if x not in sender_receiver_emails]

In [27]:
emails['Sender_Receiver_Emails'] = emails.apply(get_sender_receiver_emails,axis=1)
emails['Sender_Emails_All'] = emails.apply(get_sender_emails,axis=1)

# emails['Sender_Emails_Filtered'] = emails.apply(remove_same_emails,axis=1)

In [34]:
len(list(emails.Sender_Receiver_Emails)[-1])

6

In [35]:
emails.to_csv('human_validation_with_relevent.csv')

In [ ]:
emails_small = emails.drop(columns = ['Sender_Emails_All','Sender_Emails_Filtered'])

In [ ]:
emails_small.to_csv('gs://user-scripts-msca310019-capstone-49b3/data/20231008_Emails_with_Previous_as_List.csv')

emails_small.to_csv('gs://user-scripts-msca310019-capstone-49b3/data/20231019_Emails_with_Sender_Reply.csv')


In [20]:
emails = emails.drop(columns=['Sender_Emails_All'])

In [21]:
emails.to_csv('gs://user-scripts-msca310019-capstone-49b3/data/20231008_Emails_with_All_Previous.csv')

In [ ]:
def email_retreival(sender_id,recipients,df):
    sender_df = df[df.sender == sender_id]
    print(sender_df)
            